## Libraries Import

In [1]:
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading

In [2]:
df = pd.read_csv('datanya.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 20 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Employee_ID                  100000 non-null  int64  
 1   Department                   100000 non-null  object 
 2   Gender                       100000 non-null  object 
 3   Age                          100000 non-null  int64  
 4   Job_Title                    100000 non-null  object 
 5   Hire_Date                    100000 non-null  object 
 6   Years_At_Company             100000 non-null  int64  
 7   Education_Level              100000 non-null  object 
 8   Performance_Score            100000 non-null  int64  
 9   Monthly_Salary               100000 non-null  float64
 10  Work_Hours_Per_Week          100000 non-null  int64  
 11  Projects_Handled             100000 non-null  int64  
 12  Overtime_Hours               100000 non-null  int64  
 13  

In [4]:
df.head()

,Employee_ID,Department,Gender,Age,Job_Title,Hire_Date,Years_At_Company,Education_Level,Performance_Score,Monthly_Salary,Work_Hours_Per_Week,Projects_Handled,Overtime_Hours,Sick_Days,Remote_Work_Frequency,Team_Size,Training_Hours,Promotions,Employee_Satisfaction_Score,Resigned
0,1,IT,Male,55,Specialist,2022-01-19 08:03:05.556036,2,High School,5,6750.0,33,32,22,2,0,14,66,0,2.63,False
1,2,Finance,Male,29,Developer,2024-04-18 08:03:05.556036,0,High School,5,7500.0,34,34,13,14,100,12,61,2,1.72,False
2,3,Finance,Male,55,Specialist,2015-10-26 08:03:05.556036,8,High School,3,5850.0,37,27,6,3,50,10,1,0,3.17,False
3,4,Customer Support,Female,48,Analyst,2016-10-22 08:03:05.556036,7,Bachelor,2,4800.0,52,10,28,12,100,10,0,1,1.86,False
4,5,Engineering,Female,36,Analyst,2021-07-23 08:03:05.556036,3,Bachelor,2,4800.0,38,11,29,13,100,15,9,1,1.25,False


In [5]:
df.set_index("Employee_ID", inplace=True)

In [6]:
df_copy = df.copy()

## Exploratory Data Analysis 1

1. Ditemukan kolum menarik, yaitu 'Performance_Score'

In [7]:
df['Performance_Score'].value_counts()

1    20120
2    20013
3    19999
4    19940
5    19928
Name: Performance_Score, dtype: int64

Untuk dataset ini kita bisa membandingkan hasil clustering dengan kolum ini. Bisa saja hasil clustering kita dijadikan uji akurasi. Pas Clustering nanti kolum ini di drop saja.

2. Uji korelasi data numerik

In [8]:
things = ['object', 'boolean']
actual_num = df.select_dtypes(exclude=things)

In [9]:
actual_num.head()

,Age,Years_At_Company,Performance_Score,Monthly_Salary,Work_Hours_Per_Week,Projects_Handled,Overtime_Hours,Sick_Days,Remote_Work_Frequency,Team_Size,Training_Hours,Promotions,Employee_Satisfaction_Score
Employee_ID,,,,,,,,,,,,,
1,55,2,5,6750.0,33,32,22,2,0,14,66,0,2.63
2,29,0,5,7500.0,34,34,13,14,100,12,61,2,1.72
3,55,8,3,5850.0,37,27,6,3,50,10,1,0,3.17
4,48,7,2,4800.0,52,10,28,12,100,10,0,1,1.86
5,36,3,2,4800.0,38,11,29,13,100,15,9,1,1.25


In [10]:
target = df['Resigned']

In [11]:
p_values = []
interpretasi = []
cols = []

for col in actual_num.columns:
  corr_coef, p_value = stats.kendalltau(actual_num[col], target)

  p_values.append(p_value)
  cols.append(col)

  if p_value < 0.05:
    interpretasi.append('Correlated')
  else :
    interpretasi.append('No Correlation')

pd.DataFrame({'Column Names':cols,
              'Result': interpretasi})

,Column Names,Result
0,Age,No Correlation
1,Years_At_Company,No Correlation
2,Performance_Score,No Correlation
3,Monthly_Salary,No Correlation
4,Work_Hours_Per_Week,No Correlation
5,Projects_Handled,No Correlation
6,Overtime_Hours,No Correlation
7,Sick_Days,No Correlation
8,Remote_Work_Frequency,Correlated
9,Team_Size,No Correlation


Tidak berkorelasi guyss